# Markov Chain Models in Neo4j with Random Walks

This notebook implements various random walk models using Markov Chains in Neo4j.

In [37]:
from py2neo import Graph
import numpy as np
import random

# Connect to Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))


## Function to Create Markov Chain in Neo4j

In [38]:
def create_markov_chain(graph, transition_matrix):
    """
    Creates a Markov Chain in Neo4j from a transition matrix.
    """
    N = len(transition_matrix)
    
    # Clear previous data
    graph.run("MATCH (n) DETACH DELETE n")

    # Create state nodes
    for i in range(N):
        graph.run("CREATE (:State {name: $name, current: 0})", parameters={"name": f"S{i}"})

    # Create transitions
    for i in range(N):
        for j in range(N):
            probability = float(transition_matrix[i][j])  # Convert numpy.float64 to Python float
            if probability > 0:
                graph.run(
                    "MATCH (a:State {name: $from_state}), (b:State {name: $to_state}) "
                    "CREATE (a)-[:TO {probability: $prob}]->(b)",
                    parameters={
                        "from_state": f"S{i}",
                        "to_state": f"S{j}",
                        "prob": probability
                    }
                )

    print("Markov Chain created.")
    print(np.array(transition_matrix))


## Random Walk Simulation

In [39]:
def simulate_random_walk(graph, start_state, steps=10):
    current_state = start_state

    for step in range(steps):
        print(f'Step {step+1}: Current State -> {current_state}')
        graph.run("MATCH (s:State) SET s.current = 0")
        graph.run("MATCH (s:State {name: $state}) SET s.current = 1", state=current_state)

        result = graph.run("MATCH (s:State {name: $state})-[r:TO]->(n) RETURN n.name AS next_state, r.probability AS prob",
                         state=current_state).data()

        if not result:
            print("No transitions available. Stopping simulation.")
            break

        next_states = [record["next_state"] for record in result]
        probabilities = [record["prob"] for record in result]
        current_state = random.choices(next_states, probabilities)[0]

    print(f'Final State after {steps} steps: {current_state}')

## Example: Balanced Random Walk

In [40]:
def balanced_random_walk(N):
    matrix = np.zeros((N, N))
    for i in range(N):
        if i > 0:
            matrix[i][i - 1] = 0.5
        if i < N - 1:
            matrix[i][i + 1] = 0.5
    return matrix

N = 5
balanced_matrix = balanced_random_walk(N)
balanced_matrix

array([[0. , 0.5, 0. , 0. , 0. ],
       [0.5, 0. , 0.5, 0. , 0. ],
       [0. , 0.5, 0. , 0.5, 0. ],
       [0. , 0. , 0.5, 0. , 0.5],
       [0. , 0. , 0. , 0.5, 0. ]])

In [42]:
create_markov_chain_from_matrix(graph, balanced_matrix)

TypeError: Values of type <class 'numpy.float64'> are not supported